In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import fiona

pd.options.display.max_columns = None 

# set working directory
os.chdir('/Users/ichittumuri/Desktop/MINES/COGCC-Risk-Analysis/Data')

In [2]:
# Load Data
spills = gpd.read_file('flowlines_with_spills.geojson')
no_spills = gpd.read_file('flowlines_without_spills.geojson')

# Data Cleaning


## No Spills

In [3]:
print(no_spills.info())
print(no_spills.shape)

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 14853 entries, 0 to 14852
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ACTIONDESCRIPTION  2807 non-null   object        
 1   BEDDINGMATERIAL    7684 non-null   object        
 2   COMPANY_NAME       14853 non-null  object        
 3   CONSTRUCTDATE      14853 non-null  datetime64[ns]
 4   Diam_in            14853 non-null  float64       
 5   ENDLAT             14853 non-null  float64       
 6   ENDLONG            14853 non-null  float64       
 7   ENTIRELINEREMOVED  1764 non-null   object        
 8   FLOWLINEACTION     10495 non-null  object        
 9   FLOWLINEID         14843 non-null  float64       
 10  Fluid              14853 non-null  object        
 11  LOCATIONTYPE       14853 non-null  object        
 12  LOCATION_ID        14841 non-null  float64       
 13  Length_ft          14853 non-null  float64       
 14

In [4]:
no_spills.describe()

,CONSTRUCTDATE,Diam_in,ENDLAT,ENDLONG,FLOWLINEID,LOCATION_ID,Length_ft,MAXOPPRESSURE,OPERATOR_NUM,SHAPE_Length,STARTLAT,STARTLOCATIONID,STARTLONG
count,14853,14853.000000,14853.000000,14853.000000,14843.000000,14841.000000,14853.000000,6088.000000,14853.000000,14853.000000,14853.000000,14562.000000,14853.000000
mean,2000-08-19 00:18:54.316299648,3.034890,39.976417,-104.574987,469333.172337,405606.214541,4919.429999,466.691524,36935.902713,1499.991087,39.976332,348057.459346,-104.575311
min,1900-01-11 00:00:00,0.000000,37.010428,-109.045542,455152.000000,159601.000000,2.200000,0.000000,710.000000,0.671472,36.993609,159652.000000,-109.049983
25%,1992-01-01 00:00:00,2.000000,39.999330,-104.920760,464606.500000,328266.000000,664.170000,40.000000,10633.000000,202.925598,39.998776,317506.000000,-104.926462
50%,2005-12-09 00:00:00,2.000000,40.157850,-104.683675,469102.000000,434300.000000,1295.080000,150.000000,10699.000000,395.337163,40.158210,328918.000000,-104.685410
75%,2011-01-23 00:00:00,3.000000,40.357490,-104.226813,474949.500000,464139.000000,2264.010000,500.000000,47120.000000,692.400139,40.357510,338345.500000,-104.227525
max,2020-06-01 00:00:00,16.000000,40.992630,-102.045863,484080.000000,483552.000000,152023.900000,4700.000000,200077.000000,46342.379790,40.996000,484366.000000,-102.046467
std,NaN,2.201265,0.822864,1.314215,6440.218693,66794.604811,19078.810361,817.795615,35550.631440,5814.379003,0.822717,50345.747211,1.314151


### Get line age from construction date

In [5]:
# Verify CONSTRUCTDATE is datetime type; convert if necessary
no_spills['CONSTRUCTDATE'] = pd.to_datetime(no_spills['CONSTRUCTDATE'])

# Calculate line_age
# Today's date
today = pd.Timestamp.now()

# Calculate the difference in years
no_spills['line_age_yr'] = (today - no_spills['CONSTRUCTDATE']).dt.days / 365.25

# Display the updated GeoDataFrame to verify 'line_age' column
print(no_spills[['CONSTRUCTDATE', 'line_age_yr']])

      CONSTRUCTDATE  line_age_yr
0        1983-11-09    40.468172
1        1983-12-07    40.391513
2        2006-06-05    17.897331
3        2007-04-08    17.056810
4        2005-05-10    18.967830
...             ...          ...
14848    2001-10-02    22.570842
14849    2008-02-22    16.180698
14850    2007-08-05    16.731006
14851    1998-09-27    25.585216
14852    1983-10-21    40.520192

[14853 rows x 2 columns]


In [6]:
# Print the list of column names
column_names = no_spills.columns.tolist()

print(column_names)

['ACTIONDESCRIPTION', 'BEDDINGMATERIAL', 'COMPANY_NAME', 'CONSTRUCTDATE', 'Diam_in', 'ENDLAT', 'ENDLONG', 'ENTIRELINEREMOVED', 'FLOWLINEACTION', 'FLOWLINEID', 'Fluid', 'LOCATIONTYPE', 'LOCATION_ID', 'Length_ft', 'MAXOPPRESSURE', 'Material', 'OPERATOR_NUM', 'Operator', 'PIPEMATERIAL', 'RECEIVE_DATE', 'SHAPE_Length', 'STARTLAT', 'STARTLOCATIONID', 'STARTLONG', 'Status', 'TYPEOFFLUIDTRANS', 'geometry', 'line_age_yr']


In [7]:
# List of columns to be removed
columns_to_remove = [
    "BEDDINGMATERIAL", "COMPANY_NAME", "ENDLAT", "ENDLONG", "ENTIRELINEREMOVED",
    "RECEIVE_DATE", "STARTLAT", "STARTLOCATIONID", "STARTLONG","ACTIONDESCRIPTION","Operator",
    "TYPEOFFLUIDTRANS", "PIPEMATERIAL", "CONSTRUCTDATE"
]

# Drop the columns from the DataFrame
no_spills = no_spills.drop(columns=columns_to_remove)

In [8]:
# drop any NAs
no_spills = no_spills.dropna()

In [9]:
# Reorder df
new_order = ['OPERATOR_NUM', 'FLOWLINEID', 'LOCATION_ID', 'Status', 'FLOWLINEACTION', 'LOCATIONTYPE', 
             'Fluid', 'Material', 'Diam_in', 'Length_ft', 'MAXOPPRESSURE', 'SHAPE_Length', 'line_age_yr', 'geometry'] 
no_spills = no_spills[new_order]
no_spills

,OPERATOR_NUM,FLOWLINEID,LOCATION_ID,Status,FLOWLINEACTION,LOCATIONTYPE,Fluid,Material,Diam_in,Length_ft,MAXOPPRESSURE,SHAPE_Length,line_age_yr,geometry
1,10633,470445.0,470443.0,Active,Out of Service,Production Facilities,Multiphase,Carbon Steel,2.00,1025.98,250.0,312.594254,40.391513,"MULTILINESTRING ((507681.553 4440214.644, 5076..."
5,68710,467411.0,318620.0,Active,Pre-Abandonment Notice,Production Facilities,Crude Oil Emulsion,Fiberglass,2.00,276.03,150.0,84.099157,45.056810,"MULTILINESTRING ((497028.859 4434818.938, 4970..."
7,10459,462601.0,452637.0,Out of Service,Out of Service,Production Facilities,Multiphase,Carbon Steel,2.25,510.01,1140.0,155.389195,16.895277,"MULTILINESTRING ((496682.460 4457399.772, 4966..."
8,10459,462602.0,452637.0,Out of Service,Out of Service,Production Facilities,Multiphase,Carbon Steel,2.25,1597.85,1140.0,486.830336,13.979466,"MULTILINESTRING ((496340.456 4457384.126, 4963..."
9,10633,473671.0,336437.0,Abandoned,Out of Service,Production Facilities,Multiphase,Steel,2.00,3457.32,265.0,1053.369404,20.804928,"MULTILINESTRING ((507139.811 4448160.801, 5071..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14724,96155,456386.0,433999.0,Active,Registration,Production Facilities,Multi-Phase,Carbon Steel,3.50,1404.32,150.0,427.915127,5.875428,"MULTILINESTRING ((597095.529 4518130.982, 5970..."
14725,96155,456381.0,433999.0,Active,Registration,Production Facilities,Multi-Phase,Carbon Steel,3.50,1404.32,150.0,427.915127,5.867214,"MULTILINESTRING ((597095.529 4518130.982, 5970..."
14726,96155,456382.0,433999.0,Active,Registration,Production Facilities,Multi-Phase,Carbon Steel,3.50,1404.32,150.0,427.915127,5.875428,"MULTILINESTRING ((597095.529 4518130.982, 5970..."
14832,35080,455592.0,443145.0,Active,Registration,Production Facilities,Oil,HDPE,3.50,1175.83,40.0,358.332547,6.450376,"MULTILINESTRING ((637207.682 4380630.737, 6371..."


### Consolidate variables uniformly

In [10]:
# List of specific columns for which to print unique values
columns_to_check = ['Status', 'FLOWLINEACTION', 'LOCATIONTYPE', 'Fluid', 'Material']

# Create a dictionary to store unique values for each column
unique_values_dict = {}

# Loop through the specified columns and store unique values in the dictionary
for column in columns_to_check:
    if column in no_spills.columns:  # Check if the column exists in the DataFrame
        unique_values = no_spills[column].unique().tolist()  # Convert numpy array to list
        unique_values_dict[column] = unique_values
    else:
        unique_values_dict[column] = "Column not found in DataFrame."

# Print each column's unique values list
for column, values in unique_values_dict.items():
    print(f"Unique values in {column}: {values}")

Unique values in Status: ['Active', 'Out of Service', 'Abandoned', 'ACTIVE', 'Inactive', 'abandoned', 'InActive', 'OOS', 'Abandoned in Place', 'ABANDONED', 'Actove', 'OutofService', 'Avtive', 'PA', 'INACTIVE', 'Out-of-Service', 'Shut in', 'Status', 'Future', 'Actve']
Unique values in FLOWLINEACTION: ['Out of Service', 'Pre-Abandonment Notice', 'Abandonment Verification', 'Realignment', 'Registration', 'Removed From Service', 'Abandonment']
Unique values in LOCATIONTYPE: ['Production Facilities', 'Well Site', 'Manifold', 'Compressor Station', 'Gathering Line', 'Crude Oil Transfer Line', 'Produced Water Transfer System']
Unique values in Fluid: ['Multiphase', 'Crude Oil Emulsion', 'Emulsion', 'NATUAL GAS', 'CO2/Produced Water', 'Oil', 'Co2/Prod Water', 'Natural Gas', 'Gas', 'Produced Water', 'MULTIPHASE', 'CRUDE OIL EMULSION', 'Water', 'Condensate', 'Crude Oil', 'Unprocessed Production Fluids', 'NATURAL GAS', 'Multi-Phase', 'CO2/Prod Water', 'CO2Produced Water', 'CO2/Produced Wtaer', 'C0

In [11]:
status_mapping = {
    'Active': 'Active', 'ACTIVE': 'Active', 'Actove': 'Active', 'Avtive': 'Active', 'Actve': 'Active',
    'Out of Service': 'Out of Service', 'OOS': 'Out of Service', 'OutofService': 'Out of Service', 'Out-of-Service': 'Out of Service',
    'Abandoned': 'Abandoned', 'abandoned': 'Abandoned', 'Abandoned in Place': 'Abandoned', 'ABANDONED': 'Abandoned',
    'Inactive': 'Inactive', 'InActive': 'Inactive', 'INACTIVE': 'Inactive',
    'PA': 'Pending Analysis', 'Shut in': 'Shut In',
    'Status': 'Unknown', 'Future': 'Future'
}
no_spills['Status'] = no_spills['Status'].replace(status_mapping)
print(no_spills['Status'].unique())


flowlineaction_mapping = {
    'Out of Service': 'Out of Service', 'Removed From Service': 'Out of Service',
    'Pre-Abandonment Notice': 'Pre-Abandonment Notice',
    'Abandonment Verification': 'Abandonment Verification',
    'Realignment': 'Realignment',
    'Registration': 'Registration',
    'Abandonment': 'Abandonment'
}
no_spills['FLOWLINEACTION'] = no_spills['FLOWLINEACTION'].replace(flowlineaction_mapping)
print(no_spills['FLOWLINEACTION'].unique())


locationtype_mapping = {
    'Production Facilities': 'Production Facilities', 'Well Site': 'Well Site', 'Manifold': 'Manifold',
    'Compressor Station': 'Compressor Station', 'Gathering Line': 'Gathering Line',
    'Crude Oil Transfer Line': 'Crude Oil Transfer Line', 'Produced Water Transfer System': 'Produced Water Transfer System'
}
no_spills['LOCATIONTYPE'] = no_spills['LOCATIONTYPE'].replace(locationtype_mapping)
print(no_spills['LOCATIONTYPE'].unique())

['Active' 'Out of Service' 'Abandoned' 'Inactive' 'Pending Analysis'
 'Shut In' 'Unknown' 'Future']
['Out of Service' 'Pre-Abandonment Notice' 'Abandonment Verification'
 'Realignment' 'Registration' 'Abandonment']
['Production Facilities' 'Well Site' 'Manifold' 'Compressor Station'
 'Gathering Line' 'Crude Oil Transfer Line'
 'Produced Water Transfer System']


In [12]:
# Update the 'Fluid' column normalization and mapping in one comprehensive block
no_spills['Fluid'] = no_spills['Fluid'].str.strip().str.title().replace({
    'Natual Gas': 'Natural Gas',  # Correct common misspelling
    'Natural Gas Production': 'Natural Gas',
    'Co2': 'Co2/Produced Water',  # Assuming Co2 implies the mixed type
    'C02/Prod Water': 'Co2/Produced Water',
    'Co2/Prod Water': 'Co2/Produced Water',
    'Co2Produced Water': 'Co2/Produced Water',
    'Co2/Produced Wtaer': 'Co2/Produced Water',
    'Gas': 'Natural Gas',  # Assuming general Gas to be Natural Gas
    'Gas, Oil And Water': 'Full Well Stream',
    'Oil': 'Crude Oil',  # Assuming Oil to be categorized as Crude Oil
    'Crude Oil': 'Crude Oil',
    'Crude Oil Emulsion': 'Crude Oil Emulsion',
    'Emulsion': 'Crude Oil Emulsion',  # To unify with Crude Oil Emulsion
    'Crude Oil Emmulsion, Water And Oil': 'Crude Oil Emulsion',
    'Crude Oil And Water Emulsion': 'Crude Oil Emulsion',
    'Oil Water Emulsion': 'Crude Oil Emulsion',
    'Oil/Water': 'Crude Oil Emulsion',
    'Oil /Water/Gas': 'Full Well Stream',
    '3 Phase': 'Multiphase',  # Assumed to mean the same
    'Multiphase': 'Multiphase',
    'Multi-Phase': 'Multiphase',
    'Multi-Phase\xa0': 'Multiphase',  # Non-breaking space issue
    'Injection Produced Water': 'Produced Water',
    'Produced Water': 'Produced Water',
    'Water': 'Produced Water',
    'Saltwater': 'Produced Water',
    'Condensate': 'Condensate',  # Depending on your classification needs
    'Liquid': 'Other',  # General term, assuming to categorize broadly
    'Unprocessed Production Fluids': 'Other',
    'Production Fluids': 'Other',
    'Full Well Stream': 'Full Well Stream',  # Assuming similar handling
    'Other': 'Other',
    'Gas,  Oil And Water' : 'Full Well Stream',
    'Natural Gas Lift': 'Natural Gas',
    'Natuarl Gas': 'Natural Gas',
    'Natural Gas High Pressure': 'Natural Gas',
    'Natural Gas Supply':'Natural Gas',
    'Crude Oill Emulsion': 'Crude Oil Emulsion',
})

# Review the changes to ensure the mappings are applied correctly
print(no_spills['Fluid'].unique())

['Multiphase' 'Crude Oil Emulsion' 'Natural Gas' 'Co2/Produced Water'
 'Crude Oil' 'Produced Water' 'Condensate' 'Other' 'Full Well Stream'
 'Mulitphase' 'Oil Water']


In [13]:
no_spills['Material'] = no_spills['Material'].str.strip().str.title().replace({
    'Fiberglass': 'Fiberglass', 
    'Fibergalss': 'Fiberglass', 
    'Fiberspar': 'Fiberglass', 
    'Fiber Glass': 'Fiberglass',
    'Carbon Steel': 'Carbon Steel', 
    'Carbon Steel - Hdpe': 'Carbon Steel/HDPE', 
    'Carbon Steel, Hdpe,Stainless Steel': 'Carbon Steel/HDPE/Stainless Steel', 
    'Carbon Steel, Hdpe, Stainless Steel': 'Carbon Steel/HDPE/Stainless Steel',
    'Carbon Steel/Stainless Steel/Hdpe': 'Carbon Steel/HDPE/Stainless Steel', 
    'Carbon Steel/Hdpe/Stainless': 'Carbon Steel/HDPE/Stainless Steel', 
    'Carbon Steel/Hdpe': 'Carbon Steel/HDPE', 
    'Satinless/Carbon Steel/Hdpe': 'Carbon Steel/HDPE/Stainless Steel',
    'Steel': 'Steel', 
    'Lined Steel': 'Steel', 
    'Coated Steel': 'Steel', 
    'Flexsteel': 'Steel', 
    'Fiber Glass And Carbon Steel': 'Fiberglass/Carbon Steel', 
    'Fiberglass And Hdpe': 'Fiberglass/HDPE',
    'Hdpe': 'HDPE', 
    'Hdpe Poly': 'HDPE', 
    'Hdpe/Steel': 'HDPE/Steel', 
    'Hdpe Lined Steel': 'HDPE/Steel',
    'Poly': 'Poly', 
    'Polyline': 'Poly', 
    'Poly & Steel': 'Poly/Steel', 
    'Polycarbonate': 'Polycarbonate', 
    'Polycarbonate/Steel': 'Polycarbonate/Steel',
    'Pvc': 'PVC', 
    'Flexspar': 'Fiberglass', 
    'Stainless': 'Steel', 
    'Stainless/Carbon Steel/Hdpe': 'Carbon Steel/HDPE/Stainless Steel',
    'Unknown': 'Unknown', 
    'Other': 'Other', 
    'Other (Poly)': 'Other', 
    'Sdr7 Polyethelyne': 'Polyethylene', 
    'Sdr 11 Poly Pipe': 'Polyethylene', 
    'Sdr 11 Poly': 'Polyethylene', 
    'Poly Pipe': 'Polyethylene', 
    'Sdr_Poly': 'Polyethylene',
    'Duplex': 'Duplex', 
    'Fplp': 'Other', 
    'Flowline': 'Other',
    'Flex Steel': 'Steel',
    'Fiberglass And Carbon Steel': 'Fiberglass/Carbon Steel', 
    'Stainless Steel': 'Steel',
    'HDPE Lined Steel': 'HDPE/Steel',
    'Fiberglass/Hdpe': 'Fiberglass/HDPE',
})

# Verify the changes by printing the unique values in the 'Material' column
print(no_spills['Material'].unique())

['Carbon Steel' 'Fiberglass' 'Steel' 'Poly' 'HDPE'
 'Carbon Steel/HDPE/Stainless Steel' 'Carbon Steel/HDPE' 'Unknown' 'Other'
 'Poly/Steel' 'Carbon Steel/Hdpe/Stainless Steel' 'Duplex'
 'Co2/Produced Water' 'PVC' 'Polycarbonate' 'Polycarbonate/Steel'
 'Carbon Steel/Stainless/Hdpe' 'Polyethylene' 'Polypropylene'
 'Fiberglass/Carbon Steel' 'Hdpe/Steel, Flexsteel' 'Fiberglass/HDPE'
 'HDPE/Steel']


In [14]:
no_spills.columns = [
    'operator_number',  # Changed from OPERATOR_NUM to operator_number
    'flowline_id',       # Changed from FLOWLINEID to flowline_id
    'location_id',       # Changed from LOCATION_ID to location_id
    'status',            # Already appropriately named but changed to lowercase
    'flowline_action',   # Changed from FLOWLINEACTION to flowline_action
    'location_type',     # Changed from LOCATIONTYPE to location_type
    'fluid',             # Already appropriately named but changed to lowercase
    'material',          # Already appropriately named but changed to lowercase
    'diameter_in',       # Changed from Diam_in to diameter_in
    'length_ft',         # Changed from Length_ft to length_ft
    'max_operating_pressure', # Changed from MAXOPPRESSURE to max_operating_pressure
    'shape_length',      # Changed from SHAPE_Length to shape_length
    'line_age_yr',
    'geometry',           # Already appropriately named but changed to lowercase
]

# Print new column names to verify
print(no_spills.columns.tolist())

['operator_number', 'flowline_id', 'location_id', 'status', 'flowline_action', 'location_type', 'fluid', 'material', 'diameter_in', 'length_ft', 'max_operating_pressure', 'shape_length', 'line_age_yr', 'geometry']


In [15]:
no_spills.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 4116 entries, 1 to 14844
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   operator_number         4116 non-null   int64   
 1   flowline_id             4116 non-null   float64 
 2   location_id             4116 non-null   float64 
 3   status                  4116 non-null   object  
 4   flowline_action         4116 non-null   object  
 5   location_type           4116 non-null   object  
 6   fluid                   4116 non-null   object  
 7   material                4116 non-null   object  
 8   diameter_in             4116 non-null   float64 
 9   length_ft               4116 non-null   float64 
 10  max_operating_pressure  4116 non-null   float64 
 11  shape_length            4116 non-null   float64 
 12  line_age_yr             4116 non-null   float64 
 13  geometry                4116 non-null   geometry
dtypes: float64(7), geome

In [16]:
# make variables integers
no_spills['diameter_in'] = no_spills['diameter_in'].astype(int)
no_spills['length_ft'] = no_spills['length_ft'].astype(int)
no_spills['max_operating_pressure'] = no_spills['max_operating_pressure'].astype(int)
no_spills['shape_length'] = no_spills['shape_length'].astype(int)
no_spills['line_age_yr'] = no_spills['line_age_yr'].astype(int)

In [17]:
# Create a new column 'Risk' in the no_spills DataFrame and set all its values to 0
no_spills['risk'] = 0

### Turning text columns into numbers

Ordinal Data: If the categories have a meaningful order or rank, they are ordinal. In such cases, assigning numbers like 1-5 makes sense as the numerical order reflects the order of the categories. For example, ratings from 'Very Poor' to 'Excellent' could be encoded as 1 through 5.

Nominal Data: If the categories do not have an inherent order, they are nominal. For such data, using numeric encoding (like 1, 2, 3, etc.) can inadvertently imply a hierarchy or order that doesn't actually exist, which might lead to misleading results in your model. Instead, you should use one-hot encoding (also known as dummy variables).

In [18]:
no_spills_transformed = pd.get_dummies(no_spills, columns=['operator_number', 'flowline_id', 'location_id', 'status', 'flowline_action', 'location_type', 'fluid', 'material'], drop_first=True)

# List of original columns that are not one-hot encoded
original_columns = no_spills.columns.tolist()

# List of new columns that are the result of get_dummies
new_dummy_columns = [col for col in no_spills_transformed.columns if col not in original_columns]

# Convert only the dummy columns to int
no_spills_transformed[new_dummy_columns] = no_spills_transformed[new_dummy_columns].astype(int)

In [19]:
no_spills_transformed

diameter_in  length_ft  max_operating_pressure  shape_length  \
1                2       1025                     250           312   
5                2        276                     150            84   
7                2        510                    1140           155   
8                2       1597                    1140           486   
9                2       3457                     265          1053   
...            ...        ...                     ...           ...   
14724            3       1404                     150           427   
14725            3       1404                     150           427   
14726            3       1404                     150           427   
14832            3       1175                      40           358   
14844            3        199                    1000            60   

       line_age_yr                                           geometry  risk  \
1               40  MULTILINESTRING ((507681.553 4440214.644, 5076...     0   
5               45  MULTILINESTRING ((497028.859 4434818.938, 4970...     0   
7               16  MULTILINESTRING ((496682.460 4457399.772, 4966...     0   
8               13  MULTILINESTRING ((496340.456 4457384.126, 4963...     0   
9               20  MULTILINESTRING ((507139.811 4448160.801, 5071...     0   
...            ...                                                ...   ...   
14724            5  MULTILINESTRING ((597095.529 4518130.982, 5970...     0   
14725            5  MULTILINESTRING ((597095.529 4518130.982, 5970...     0   
14726            5  MULTILINESTRING ((597095.529 4518130.982, 5970...     0   
14832            6  MULTILINESTRING ((637207.682 4380630.737, 6371...     0   
14844           41  MULTILINESTRING ((496551.343 4443672.866, 4964...     0   

       operator_number_3250  operator_number_6720  operator_number_7125  \
1                         0                     0                     0   
5                         0                     0                     0   
7                         0                     0                     0   
8                         0                     0                     0   
9                         0                     0                     0   
...                     ...                   ...                   ...   
14724                     0                     0                     0   
14725                     0                     0                     0   
14726                     0                     0                     0   
14832                     0                     0                     0   
14844                     0                     0                     0   

       operator_number_7800  operator_number_8960  operator_number_10000  \
1                         0                     0                      0   
5                         0                     0                      0   
7                         0                     0                      0   
8                         0                     0                      0   
9                         0                     0                      0   
...                     ...                   ...                    ...   
14724                     0                     0                      0   
14725                     0                     0                      0   
14726                     0                     0                      0   
14832                     0                     0                      0   
14844                     0                     0                      0   

       operator_number_10017  operator_number_10071  operator_number_10085  \
1                          0                      0                      0   
5                          0                      0                      0   
7                          0                      0                      0   
8                          0                      0                      0   
9

## Spills

In [20]:
print(spills.info())
print(spills.shape)

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 40 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   ACTIONDESCRIPTION         5 non-null      object        
 1   BEDDINGMATERIAL           71 non-null     object        
 2   COMPANY_NAME              84 non-null     object        
 3   CONSTRUCTDATE             84 non-null     datetime64[ns]
 4   Detailed Root Cause Type  72 non-null     object        
 5   Diam_in                   84 non-null     float64       
 6   ENDLAT                    84 non-null     float64       
 7   ENDLONG                   84 non-null     float64       
 8   ENTIRELINEREMOVED         1 non-null      object        
 9   FLOWLINEACTION            66 non-null     object        
 10  FLOWLINEID                83 non-null     float64       
 11  Fluid                     84 non-null     object        
 12  LOCATIONTYPE    

In [21]:
spills.describe()

,CONSTRUCTDATE,Diam_in,ENDLAT,ENDLONG,FLOWLINEID,LOCATION_ID,Lat,Length_ft,Long,MAXOPPRESSURE,OPERATOR_NUM,RECEIVE_DATE,SHAPE_Length,STARTLAT,STARTLOCATIONID,STARTLONG,trkg_num
count,84,84.000000,84.000000,84.000000,83.00000,84.000000,84.000000,84.000000,84.000000,41.000000,84.000000,84,84.000000,84.000000,83.000000,84.000000,8.400000e+01
mean,1995-09-12 16:34:17.142857088,4.352679,40.136602,-104.493780,472602.13253,389660.476190,40.140161,21823.340714,-104.489816,410.804878,39992.571429,2020-02-17 15:27:46.206321408,6650.083840,40.136836,360151.132530,-104.494600,4.023819e+08
min,1955-04-29 00:00:00,0.000000,37.106749,-108.062538,461137.00000,310063.000000,37.093239,107.770000,-108.063630,0.000000,8960.000000,2019-01-31 08:57:57.543000,32.834947,37.101221,307094.000000,-108.058653,4.017021e+08
25%,1976-11-16 18:00:00,2.000000,40.017682,-104.915835,467764.00000,317957.250000,40.017332,1118.775000,-104.914936,28.000000,10633.000000,2019-09-04 13:59:46.060000,340.945969,40.019395,318133.500000,-104.916080,4.019914e+08
50%,2000-09-13 12:00:00,3.000000,40.113755,-104.705808,474775.00000,428806.000000,40.113877,2166.010000,-104.705578,140.000000,46290.000000,2019-11-21 13:58:21.900000,659.962512,40.114540,327390.000000,-104.702951,4.022982e+08
75%,2013-10-29 00:00:00,6.000000,40.386021,-103.849656,476061.50000,439468.750000,40.385394,6605.540000,-103.810861,700.000000,46290.000000,2020-11-17 00:57:55.310499840,2012.563421,40.391077,431782.000000,-103.850427,4.029230e+08
max,2018-12-14 00:00:00,12.750000,40.851857,-102.083250,482303.00000,480689.000000,40.859583,141290.450000,-102.083057,1800.000000,100322.000000,2022-09-07 12:08:37.853000,43053.037992,40.853333,482187.000000,-102.080785,4.032349e+08
std,NaN,3.056949,0.523972,1.050751,5289.74121,66480.136341,0.530457,46037.498444,1.053401,491.096183,31923.495573,NaN,14028.568590,0.524026,55569.733297,1.050971,4.539768e+05


### Get line age from construction date

In [22]:
# Verify CONSTRUCTDATE is datetime type; convert if necessary
spills['CONSTRUCTDATE'] = pd.to_datetime(spills['CONSTRUCTDATE'])

# Calculate line_age
# Today's date
today = pd.Timestamp.now()

# Calculate the difference in years
spills['line_age_yr'] = (today - spills['CONSTRUCTDATE']).dt.days / 365.25

# Display the updated GeoDataFrame to verify 'line_age' column
print(spills[['CONSTRUCTDATE', 'line_age_yr']])

   CONSTRUCTDATE  line_age_yr
0     2014-09-25     9.590691
1     2014-09-25     9.590691
2     2002-10-01    21.574264
3     2013-10-29    10.496920
4     2013-10-29    10.496920
..           ...          ...
79    1975-12-27    48.336756
80    1976-06-29    47.830253
81    1976-08-23    47.679671
82    2008-11-24    15.425051
83    1977-04-15    47.036277

[84 rows x 2 columns]


In [23]:
# Print the list of column names
column_names = spills.columns.tolist()

print(column_names)  

['ACTIONDESCRIPTION', 'BEDDINGMATERIAL', 'COMPANY_NAME', 'CONSTRUCTDATE', 'Detailed Root Cause Type', 'Diam_in', 'ENDLAT', 'ENDLONG', 'ENTIRELINEREMOVED', 'FLOWLINEACTION', 'FLOWLINEID', 'Fluid', 'LOCATIONTYPE', 'LOCATION_ID', 'Lat', 'Length_ft', 'Long', 'MAXOPPRESSURE', 'Material', 'Metallic?', 'OPERATOR_NUM', 'Operator', 'Operator Name', 'PIPEMATERIAL', 'Preventative Measure', 'RECEIVE_DATE', 'Root Cause', 'Root Cause Type', 'SHAPE_Length', 'STARTLAT', 'STARTLOCATIONID', 'STARTLONG', 'Spill Type', 'Spill_Desc', 'Status', 'TYPEOFFLUIDTRANS', 'facility_status', 'facility_type', 'trkg_num', 'geometry', 'line_age_yr']


In [24]:
# List of columns to be removed
columns_to_remove = [
    "ACTIONDESCRIPTION", "BEDDINGMATERIAL", "COMPANY_NAME", "CONSTRUCTDATE", "ENDLAT",
    "ENDLONG", "ENTIRELINEREMOVED", "Lat", "Long","Operator","Operator Name", "Spill Type",
    "STARTLAT", "STARTLOCATIONID", "STARTLONG", "trkg_num", "Root Cause", "Detailed Root Cause Type", "Metallic?", 
    "Preventative Measure","Spill_Desc", "TYPEOFFLUIDTRANS", "facility_status", "facility_type", "PIPEMATERIAL", "RECEIVE_DATE"
]

# Drop the columns from the DataFrame
spills = spills.drop(columns=columns_to_remove)

In [25]:
# Print the list of column names
column_names = spills.columns.tolist()

print(column_names)  

['Diam_in', 'FLOWLINEACTION', 'FLOWLINEID', 'Fluid', 'LOCATIONTYPE', 'LOCATION_ID', 'Length_ft', 'MAXOPPRESSURE', 'Material', 'OPERATOR_NUM', 'Root Cause Type', 'SHAPE_Length', 'Status', 'geometry', 'line_age_yr']


In [26]:
# Reorder df
new_order = ['OPERATOR_NUM', 'FLOWLINEID', 'LOCATION_ID', 'Status', 'FLOWLINEACTION', 'LOCATIONTYPE', 
             'Fluid', 'Material', 'Diam_in', 'Length_ft', 'MAXOPPRESSURE', 'SHAPE_Length', 'line_age_yr',
              'Root Cause Type', 'geometry'] 
spills = spills[new_order]
spills

,OPERATOR_NUM,FLOWLINEID,LOCATION_ID,Status,FLOWLINEACTION,LOCATIONTYPE,Fluid,Material,Diam_in,Length_ft,MAXOPPRESSURE,SHAPE_Length,line_age_yr,Root Cause Type,geometry
0,96155,465034.0,433131.0,Active,Registration,Well Site,Crude Oil,Carbon Steel,12.375,141290.45,1200.0,43053.037992,9.590691,Corrosion,"MULTILINESTRING ((595672.003 4517988.031, 5956..."
1,96155,465034.0,433131.0,Active,Registration,Well Site,Crude Oil,Carbon Steel,12.375,141290.45,1200.0,43053.037992,9.590691,Corrosion,"MULTILINESTRING ((595672.003 4517988.031, 5956..."
2,10112,468013.0,467691.0,Active,None,Production Facilities,Multiphase,Carbon Steel,3.000,1967.95,100.0,599.638869,21.574264,Corrosion,"MULTILINESTRING ((579396.131 4497273.237, 5793..."
3,96155,467764.0,431782.0,Active,Registration,Produced Water Transfer System,Natural Gas,Carbon Steel,6.625,129999.02,550.0,39612.461236,10.496920,Unknown,"MULTILINESTRING ((593633.376 4516002.016, 5936..."
4,96155,467764.0,431782.0,Active,Registration,Produced Water Transfer System,Natural Gas,Carbon Steel,6.625,129999.02,550.0,39612.461236,10.496920,Incorrect Operation,"MULTILINESTRING ((593633.376 4516002.016, 5936..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,52530,470594.0,451015.0,Active,Registration,Well Site,"Oil, Gas, Water",Poly,2.000,524.75,NaN,159.878478,48.336756,Excavation Damage,"MULTILINESTRING ((495179.357 4469147.980, 4951..."
80,46290,476027.0,318226.0,Active,None,Production Facilities,Multiphase,Fiberglass,3.000,433.61,NaN,132.113052,47.830253,Unknown,"MULTILINESTRING ((511486.000 4440071.988, 5114..."
81,46290,478688.0,317631.0,Active,Registration,Production Facilities,Multiphase,Carbon Steel,12.750,6311.02,14.0,1922.833152,47.679671,Unknown,"MULTILINESTRING ((509695.403 4437513.863, 5096..."
82,10699,472933.0,337427.0,ACTIVE,Registration,Production Facilities,Natural Gas,HDPE,2.000,1115.25,NaN,340.053258,15.425051,"Pipe, Weld, Joint Failure","MULTILINESTRING ((749596.141 4419849.037, 7495..."


In [27]:
spills.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   OPERATOR_NUM     84 non-null     int64   
 1   FLOWLINEID       83 non-null     float64 
 2   LOCATION_ID      84 non-null     float64 
 3   Status           84 non-null     object  
 4   FLOWLINEACTION   66 non-null     object  
 5   LOCATIONTYPE     84 non-null     object  
 6   Fluid            84 non-null     object  
 7   Material         84 non-null     object  
 8   Diam_in          84 non-null     float64 
 9   Length_ft        84 non-null     float64 
 10  MAXOPPRESSURE    41 non-null     float64 
 11  SHAPE_Length     84 non-null     float64 
 12  line_age_yr      84 non-null     float64 
 13  Root Cause Type  84 non-null     object  
 14  geometry         84 non-null     geometry
dtypes: float64(7), geometry(1), int64(1), object(6)
memory usage: 10.0+ KB


### Consolidate variables uniformly

In [28]:
# List of specific columns for which to print unique values
columns_to_check = ['Status', 'FLOWLINEACTION', 'LOCATIONTYPE', 'Fluid', 'Material', 'Root Cause Type']

# Create a dictionary to store unique values for each column
unique_values_dict = {}

# Loop through the specified columns and store unique values in the dictionary
for column in columns_to_check:
    if column in spills.columns:  # Check if the column exists in the DataFrame
        unique_values = spills[column].unique().tolist()  # Convert numpy array to list
        unique_values_dict[column] = unique_values
    else:
        unique_values_dict[column] = "Column not found in DataFrame."

# Print each column's unique values list
for column, values in unique_values_dict.items():
    print(f"Unique values in {column}: {values}")

Unique values in Status: ['Active', 'ACTIVE', 'Out Of Service', 'Out of Service', 'abandoned']
Unique values in FLOWLINEACTION: ['Registration', None, 'Abandonment', 'Out of Service', 'Realignment', 'Pre-Abandonment Notice']
Unique values in LOCATIONTYPE: ['Well Site', 'Production Facilities', 'Produced Water Transfer System', 'Gathering Line', 'Manifold', 'Compressor Station']
Unique values in Fluid: ['Crude Oil', 'Multiphase', 'Natural Gas', 'Crude Oil Emulsion', 'Produced Water', 'PRODUCED WATER', 'Oil', 'Liquid', 'Natural Gas Production', 'Multi-Phase\xa0', 'Oil and water', '3 Phase', 'Oil, Gas, Water']
Unique values in Material: ['Carbon Steel', 'HDPE', 'LINED STEEL', 'Fiberglass', 'Steel', 'HDPE Poly', 'Other (Poly)', 'Other', 'FPLP', 'steel', 'Composite HDPE', 'PVC', 'Poly']
Unique values in Root Cause Type: ['Corrosion', 'Unknown', 'Incorrect Operation', 'Equipment Failure', 'Other Outside Force Damage', 'Natural Force Damage', 'Pipe, Weld, or Joint Failure', 'Excavation Damage

In [29]:
# Define the mapping dictionary for the 'Status' column
status_mapping = {
    'ACTIVE': 'Active',           # Normalize to 'Active'
    'Active': 'Active',           # No change needed, already in desired format
    'Out Of Service': 'Out of Service', # Normalize case and spacing
    'Out of Service': 'Out of Service', # No change needed
    'abandoned': 'Abandoned'      # Capitalize to 'Abandoned'
}

# Apply the mapping to the 'Status' column
spills['Status'] = spills['Status'].replace(status_mapping)
print(spills['Status'].unique())

# Define the mapping dictionary for the 'FLOWLINEACTION' column
flowlineaction_mapping = {
    'Registration': 'Registration',  # No change needed
    None: 'Unknown',                # Mapping None to 'Unknown' (or you could leave it as None if preferred)
    'Abandonment': 'Abandonment',   # No change needed
    'Out of Service': 'Out of Service', # Ensure consistent formatting if there's case variation elsewhere
    'Realignment': 'Realignment',   # No change needed
    'Pre-Abandonment Notice': 'Pre-Abandonment Notice' # No change needed
}

# Apply the mapping to the 'FLOWLINEACTION' column
spills['FLOWLINEACTION'] = spills['FLOWLINEACTION'].replace(flowlineaction_mapping)
print(spills['FLOWLINEACTION'].unique())

['Active' 'Out of Service' 'Abandoned']
['Registration' 'Unknown' 'Abandonment' 'Out of Service' 'Realignment'
 'Pre-Abandonment Notice']


In [30]:
# Define the mapping dictionary for the 'Fluid' column, aligning with the provided categories
fluid_mapping = {
    'Crude Oil': 'Crude Oil',                         
    'Multiphase': 'Multiphase',                       
    'Natural Gas': 'Natural Gas',                     
    'Crude Oil Emulsion': 'Crude Oil Emulsion',       
    'Produced Water': 'Produced Water',               
    'PRODUCED WATER': 'Produced Water',               
    'Oil': 'Crude Oil',                               # Mapping general oil as Crude Oil
    'Liquid': 'Other',                                # Mapping generic liquids to 'Other'
    'Natural Gas Production': 'Natural Gas',          # Simplifying to 'Natural Gas'
    'Multi-Phase\xa0': 'Multiphase',                  # Standardizing 'Multiphase'
    'Oil and water': 'Oil Water',                     # Standardizing as 'Oil Water'
    '3 Phase': 'Multiphase',                          # Mapping to 'Multiphase'
    'Oil, Gas, Water': 'Oil Water',                   # Assuming mix of oil and water primarily
    'Mulitphase': 'Multiphase',                       # Correcting typo and standardizing
    'Co2/Produced Water': 'Co2/Produced Water',       # No change needed, already a standard category
    'Condensate': 'Condensate',                       # Adding as its own category
    'Full Well Stream': 'Full Well Stream'            # Adding as its own category
}

# Apply the mapping to the 'Fluid' column
spills['Fluid'] = spills['Fluid'].replace(fluid_mapping)
print(spills['Fluid'].unique())

['Crude Oil' 'Multiphase' 'Natural Gas' 'Crude Oil Emulsion'
 'Produced Water' 'Other' 'Oil Water']


In [31]:
# Define the mapping dictionary for the 'Material' column
material_mapping = {
    'Carbon Steel': 'Carbon Steel', 
    'HDPE': 'HDPE', 
    'LINED STEEL': 'Steel',          # Assuming Lined Steel is generally categorized as Steel
    'Fiberglass': 'Fiberglass', 
    'Steel': 'Steel', 
    'HDPE Poly': 'HDPE',             # Standardizing as HDPE
    'Other (Poly)': 'Other',         # Simplifying to 'Other'
    'Other': 'Other', 
    'FPLP': 'Other',                 # FPLP is often a specific type of lining or pipe, classify as Other if not specifically relevant
    'steel': 'Steel',                # Ensuring case consistency
    'Composite HDPE': 'HDPE',        # Composite materials containing HDPE, categorized as HDPE
    'PVC': 'PVC', 
    'Poly': 'Poly', 
    'Poly/Steel': 'Poly/Steel',      # This is already in the reference list
    'Carbon Steel/HDPE/Stainless Steel': 'Carbon Steel/HDPE/Stainless Steel', 
    'Carbon Steel/HDPE': 'Carbon Steel/HDPE', 
    'Unknown': 'Unknown',            # Keep as is if applicable
    'Duplex': 'Duplex',              # Duplex typically refers to stainless steel but keeping separate as may be specific
    'Co2/Produced Water': 'Co2/Produced Water',   # Specific type of material handling, categorized separately
    'Polycarbonate': 'Polycarbonate',
    'Polycarbonate/Steel': 'Polycarbonate/Steel',
    'Polyethylene': 'Polyethylene',  # Mapping to Polyethylene where applicable
    'Polypropylene': 'Polypropylene',
    'Fiberglass/Carbon Steel': 'Fiberglass/Carbon Steel',
    'Hdpe/Steel, Flexsteel': 'HDPE/Steel'         # Assuming Hdpe/Steel includes any combination like Flexsteel
}

# Apply the mapping to the 'Material' column
spills['Material'] = spills['Material'].replace(material_mapping)
print(spills['Material'].unique())

['Carbon Steel' 'HDPE' 'Steel' 'Fiberglass' 'Other' 'PVC' 'Poly']


In [32]:
# Define the mapping dictionary for the 'Root Cause Type' column
root_cause_mapping = {
    'Corrosion': 'Corrosion', 
    'Unknown': 'Unknown', 
    'Incorrect Operation': 'Incorrect Operation', 
    'Equipment Failure': 'Equipment Failure', 
    'Other Outside Force Damage': 'Other Outside Force Damage', 
    'Natural Force Damage': 'Natural Force Damage', 
    'Pipe, Weld, or Joint Failure': 'Pipe, Weld, or Joint Failure', 
    'Excavation Damage': 'Excavation Damage', 
    'Other Outside Force': 'Other Outside Force Damage',  # Mapping to a more standardized category
    'Pipe, Weld, Joint Failure': 'Pipe, Weld, or Joint Failure'  # Ensuring consistency in naming
}

# Apply the mapping to the 'Root Cause Type' column
spills['Root Cause Type'] = spills['Root Cause Type'].replace(root_cause_mapping)
print(spills['Root Cause Type'].unique())

['Corrosion' 'Unknown' 'Incorrect Operation' 'Equipment Failure'
 'Other Outside Force Damage' 'Natural Force Damage'
 'Pipe, Weld, or Joint Failure' 'Excavation Damage']


In [33]:
# Assuming 'spills' is your DataFrame
spills.columns = [
    'operator_number',         # Changed from OPERATOR_NUM to operator_number
    'flowline_id',             # Changed from FLOWLINEID to flowline_id
    'location_id',             # Changed from LOCATION_ID to location_id
    'status',                  # Already appropriately named but changed to lowercase
    'flowline_action',         # Changed from FLOWLINEACTION to flowline_action
    'location_type',           # Changed from LOCATIONTYPE to location_type
    'fluid',                   # Already appropriately named but changed to lowercase
    'material',                # Already appropriately named but changed to lowercase
    'diameter_in',             # Changed from Diam_in to diameter_in
    'length_ft',               # Changed from Length_ft to length_ft
    'max_operating_pressure',  # Changed from MAXOPPRESSURE to max_operating_pressure
    'shape_length',            # Changed from SHAPE_Length to shape_length
    'line_age_yr',             # Changed from line_age_yr to maintain consistency in the style
    'root_cause_type',         # Added to align with the column names provided earlier
    'geometry'                 # Already appropriately named but changed to lowercase
]

# Print new column names to verify
print(spills.columns.tolist())

['operator_number', 'flowline_id', 'location_id', 'status', 'flowline_action', 'location_type', 'fluid', 'material', 'diameter_in', 'length_ft', 'max_operating_pressure', 'shape_length', 'line_age_yr', 'root_cause_type', 'geometry']


In [34]:
spills.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   operator_number         84 non-null     int64   
 1   flowline_id             83 non-null     float64 
 2   location_id             84 non-null     float64 
 3   status                  84 non-null     object  
 4   flowline_action         84 non-null     object  
 5   location_type           84 non-null     object  
 6   fluid                   84 non-null     object  
 7   material                84 non-null     object  
 8   diameter_in             84 non-null     float64 
 9   length_ft               84 non-null     float64 
 10  max_operating_pressure  41 non-null     float64 
 11  shape_length            84 non-null     float64 
 12  line_age_yr             84 non-null     float64 
 13  root_cause_type         84 non-null     object  
 14  geometry            

In [35]:
# drop any NAs
spills = spills.dropna()
spills.shape

(40, 15)

In [36]:
# make variables integers
spills['diameter_in'] = spills['diameter_in'].astype(int)
spills['length_ft'] = spills['length_ft'].astype(int)
spills['max_operating_pressure'] = spills['max_operating_pressure'].astype(int)
spills['shape_length'] = spills['shape_length'].astype(int)
spills['line_age_yr'] = spills['line_age_yr'].astype(int)

/Users/ichittumuri/Desktop/MINES/COGCC-Risk-Analysis/.venv/lib/python3.9/site-packages/geopandas/geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/ichittumuri/Desktop/MINES/COGCC-Risk-Analysis/.venv/lib/python3.9/site-packages/geopandas/geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/ichittumuri/Desktop/MINES/COGCC-Risk-Analysis/.venv/lib/python3.9/site-packages/geopandas/geodataframe.py:1525:

In [37]:
spills['risk'] = 1

/Users/ichittumuri/Desktop/MINES/COGCC-Risk-Analysis/.venv/lib/python3.9/site-packages/geopandas/geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [38]:
spills.head()

,operator_number,flowline_id,location_id,status,flowline_action,location_type,fluid,material,diameter_in,length_ft,max_operating_pressure,shape_length,line_age_yr,root_cause_type,geometry,risk
0,96155,465034.0,433131.0,Active,Registration,Well Site,Crude Oil,Carbon Steel,12,141290,1200,43053,9,Corrosion,"MULTILINESTRING ((595672.003 4517988.031, 5956...",1
1,96155,465034.0,433131.0,Active,Registration,Well Site,Crude Oil,Carbon Steel,12,141290,1200,43053,9,Corrosion,"MULTILINESTRING ((595672.003 4517988.031, 5956...",1
2,10112,468013.0,467691.0,Active,Unknown,Production Facilities,Multiphase,Carbon Steel,3,1967,100,599,21,Corrosion,"MULTILINESTRING ((579396.131 4497273.237, 5793...",1
3,96155,467764.0,431782.0,Active,Registration,Produced Water Transfer System,Natural Gas,Carbon Steel,6,129999,550,39612,10,Unknown,"MULTILINESTRING ((593633.376 4516002.016, 5936...",1
4,96155,467764.0,431782.0,Active,Registration,Produced Water Transfer System,Natural Gas,Carbon Steel,6,129999,550,39612,10,Incorrect Operation,"MULTILINESTRING ((593633.376 4516002.016, 5936...",1


### Turning text columns into numbers

In [39]:
spills.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 40 entries, 0 to 81
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   operator_number         40 non-null     int64   
 1   flowline_id             40 non-null     float64 
 2   location_id             40 non-null     float64 
 3   status                  40 non-null     object  
 4   flowline_action         40 non-null     object  
 5   location_type           40 non-null     object  
 6   fluid                   40 non-null     object  
 7   material                40 non-null     object  
 8   diameter_in             40 non-null     int64   
 9   length_ft               40 non-null     int64   
 10  max_operating_pressure  40 non-null     int64   
 11  shape_length            40 non-null     int64   
 12  line_age_yr             40 non-null     int64   
 13  root_cause_type         40 non-null     object  
 14  geometry                4

In [40]:
spills_transformed = pd.get_dummies(spills, columns=['operator_number', 'flowline_id', 'location_id', 'status', 'flowline_action', 'location_type', 'fluid', 'material', 'root_cause_type'], drop_first=True)

# List of original columns that are not one-hot encoded
original_columns = spills.columns.tolist()

# List of new columns that are the result of get_dummies
new_dummy_columns = [col for col in spills_transformed.columns if col not in original_columns]

# Convert only the dummy columns to int
spills_transformed[new_dummy_columns] = spills_transformed[new_dummy_columns].astype(int)

In [41]:
spills_transformed.head()

,diameter_in,length_ft,max_operating_pressure,shape_length,line_age_yr,geometry,risk,operator_number_10261,operator_number_10373,operator_number_10433,operator_number_10456,operator_number_10598,operator_number_10690,operator_number_17180,operator_number_46290,operator_number_95520,operator_number_96155,operator_number_98220,flowline_id_462626.0,flowline_id_463088.0,flowline_id_465034.0,flowline_id_465848.0,flowline_id_467764.0,flowline_id_468013.0,flowline_id_470771.0,flowline_id_474494.0,flowline_id_474496.0,flowline_id_475131.0,flowline_id_475134.0,flowline_id_475203.0,flowline_id_475210.0,flowline_id_475311.0,flowline_id_475369.0,flowline_id_476084.0,flowline_id_476178.0,flowline_id_476198.0,flowline_id_476289.0,flowline_id_476372.0,flowline_id_476485.0,flowline_id_478125.0,flowline_id_478674.0,flowline_id_478688.0,flowline_id_478790.0,flowline_id_480690.0,flowline_id_480883.0,flowline_id_482303.0,location_id_313937.0,location_id_316911.0,location_id_317078.0,location_id_317096.0,location_id_317357.0,location_id_317631.0,location_id_317874.0,location_id_317985.0,location_id_318425.0,location_id_318732.0,location_id_320276.0,location_id_334451.0,location_id_334538.0,location_id_336455.0,location_id_431782.0,location_id_433131.0,location_id_439195.0,location_id_440290.0,location_id_446527.0,location_id_446980.0,location_id_467691.0,location_id_476062.0,location_id_476370.0,location_id_476479.0,location_id_478672.0,location_id_478789.0,location_id_480689.0,flowline_action_Realignment,flowline_action_Registration,flowline_action_Unknown,location_type_Manifold,location_type_Produced Water Transfer System,location_type_Production Facilities,location_type_Well Site,fluid_Crude Oil Emulsion,fluid_Multiphase,fluid_Natural Gas,fluid_Other,fluid_Produced Water,material_Fiberglass,material_HDPE,material_Other,material_PVC,material_Steel,root_cause_type_Equipment Failure,root_cause_type_Excavation Damage,root_cause_type_Incorrect Operation,root_cause_type_Natural Force Damage,root_cause_type_Other Outside Force Damage,root_cause_type_Unknown
0,12,141290,1200,43053,9,"MULTILINESTRING ((595672.003 4517988.031, 5956...",1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,12,141290,1200,43053,9,"MULTILINESTRING ((595672.003 4517988.031, 5956...",1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,1967,100,599,21,"MULTILINESTRING ((579396.131 4497273.237, 5793...",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,6,129999,550,39612,10,"MULTILINESTRING ((593633.376 4516002.016, 5936...",1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
4,6,129999,550,39612,10,"MULTILINESTRING ((593633.376 4516002.016, 5936...",1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0


In [42]:
print(f"No Spills: {len(no_spills_transformed)}")
print(f"Spills: {len(spills_transformed)}")

No Spills: 4116
Spills: 40


# Download Cleaned Data

In [43]:
no_spills_transformed.to_file("no_spills_transformed.geojson", driver='GeoJSON')

In [ ]:
spills_transformed.to_file("spills_transformed.geojson", driver='GeoJSON')